In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import savefig
import cv2



C:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
num_images = 180;

dataset = []

for i in range(1, num_images+1):
    img = cv2.imread("./Augmented_Data_Hazy/" +str(i) +".png" )
    dataset.append(np.array(img))

dataset_source = np.asarray(dataset)
print(dataset_source.shape)

dataset_tar = []

for i in range(1, num_images+1):
    img = cv2.imread("./Augmented_Data_TransGT/" +str(i) +".png")    
    dataset_tar.append(np.array(img))

dataset_target = np.asarray(dataset_tar)
print(dataset_target.shape)

(180, 1200, 1600, 3)
(180, 1200, 1600, 3)


In [6]:
def autoencoder(inputs): # Undercomplete Autoencoder
    
    # Encoder
    print(inputs.shape)
    net = tf.layers.conv2d(inputs, 8, 2, activation = tf.nn.relu)
    print(net.shape)
    net = tf.layers.max_pooling2d(net, 4, 4, padding = 'same')
    print(net.shape)
    
    # Decoder
    
    net = tf.image.resize_nearest_neighbor(net, tf.constant([800, 1200]))
    print(net.shape)
    net = tf.layers.conv2d(net, 3, 2, activation = None, name = 'outputOfAuto')
    
    print(net.shape)
    net = tf.image.resize_nearest_neighbor(net, tf.constant([1200, 1600]))
    print(net.shape)
    
    return net

In [7]:
tf.reset_default_graph()
ae_inputs = tf.placeholder(tf.float32, (None, 1200, 1600, 3), name = 'inputToAuto')
print(ae_inputs.shape)
ae_target = tf.placeholder(tf.float32, (None, 1200, 1600, 3))

ae_outputs = autoencoder(ae_inputs)
lr = 0.001 # learning rate 

loss = tf.reduce_mean(tf.square(ae_outputs - ae_target))
train_op = tf.train.AdamOptimizer(learning_rate = lr).minimize(loss)
# Intialize the network 
init = tf.global_variables_initializer()

(?, 1200, 1600, 3)
(?, 1200, 1600, 3)
(?, 1199, 1599, 8)
(?, 300, 400, 8)
(?, 800, 1200, 8)
(?, 799, 1199, 3)
(?, 1200, 1600, 3)


In [8]:
batch_size = 5
epoch_num = 50

saving_path = './Autoencoder_Model/Autoencoder.ckpt'

saver_ = tf.train.Saver(max_to_keep = 3)

batch_img = dataset_source[0:batch_size]
batch_out = dataset_target[0:batch_size]

num_batches = num_images//batch_size

sess = tf.Session()
sess.run(init)

for ep in range(epoch_num):
    print("Running epoch "+ str(ep))
    batch_size = 0
    for batch_n in range(num_batches): # batches loop

        _, c = sess.run([train_op, loss], feed_dict = {ae_inputs: batch_img, ae_target: batch_out})
        print("Epoch: {} - cost = {:.5f}" .format((ep+1), c))
            
        batch_img = dataset_source[batch_size: batch_size+5]
        batch_out = dataset_target[batch_size: batch_size+5]
            
        batch_size += 5
    
    saver_.save(sess, saving_path, global_step = ep)

sess.close()

Running epoch 0
Epoch: 1 - cost = 20191.67969
Epoch: 1 - cost = 19558.99414
Epoch: 1 - cost = 18595.14258
Epoch: 1 - cost = 14440.93066
Epoch: 1 - cost = 16053.24414
Epoch: 1 - cost = 9649.83789
Epoch: 1 - cost = 18104.97461
Epoch: 1 - cost = 19639.30078
Epoch: 1 - cost = 17205.57812
Epoch: 1 - cost = 16254.72949
Epoch: 1 - cost = 14886.60547
Epoch: 1 - cost = 13588.61328
Epoch: 1 - cost = 10683.95410
Epoch: 1 - cost = 11429.89453
Epoch: 1 - cost = 6805.96094
Epoch: 1 - cost = 14447.61816
Epoch: 1 - cost = 15191.00293
Epoch: 1 - cost = 13641.35352
Epoch: 1 - cost = 12933.77148
Epoch: 1 - cost = 11316.17090
Epoch: 1 - cost = 9728.76758
Epoch: 1 - cost = 7761.70215
Epoch: 1 - cost = 7790.33936
Epoch: 1 - cost = 4587.90137
Epoch: 1 - cost = 11260.70020
Epoch: 1 - cost = 11218.73438
Epoch: 1 - cost = 10323.87891
Epoch: 1 - cost = 9956.89746
Epoch: 1 - cost = 8765.34082
Epoch: 1 - cost = 7088.28027
Epoch: 1 - cost = 5944.84717
Epoch: 1 - cost = 5432.18115
Epoch: 1 - cost = 3453.55908
Epoch:

Epoch: 8 - cost = 6905.99805
Epoch: 8 - cost = 7346.69531
Epoch: 8 - cost = 6687.00098
Epoch: 8 - cost = 5144.34912
Epoch: 8 - cost = 5414.22998
Epoch: 8 - cost = 4049.00342
Epoch: 8 - cost = 3392.77808
Epoch: 8 - cost = 7550.41211
Epoch: 8 - cost = 6923.46436
Epoch: 8 - cost = 6904.80566
Running epoch 8
Epoch: 9 - cost = 7345.56445
Epoch: 9 - cost = 6647.62012
Epoch: 9 - cost = 5140.61768
Epoch: 9 - cost = 5520.87109
Epoch: 9 - cost = 4041.04712
Epoch: 9 - cost = 3400.41943
Epoch: 9 - cost = 7563.61963
Epoch: 9 - cost = 6951.66943
Epoch: 9 - cost = 6917.43799
Epoch: 9 - cost = 7350.15332
Epoch: 9 - cost = 6735.13818
Epoch: 9 - cost = 5177.64258
Epoch: 9 - cost = 5384.95654
Epoch: 9 - cost = 4079.92529
Epoch: 9 - cost = 3365.62964
Epoch: 9 - cost = 7569.01172
Epoch: 9 - cost = 6960.98242
Epoch: 9 - cost = 6909.95215
Epoch: 9 - cost = 7347.28369
Epoch: 9 - cost = 6738.79102
Epoch: 9 - cost = 5179.91602
Epoch: 9 - cost = 5418.65088
Epoch: 9 - cost = 4083.94336
Epoch: 9 - cost = 3397.8845

Epoch: 16 - cost = 7339.62988
Epoch: 16 - cost = 6572.81934
Epoch: 16 - cost = 5072.15479
Epoch: 16 - cost = 5491.85156
Epoch: 16 - cost = 3969.52905
Epoch: 16 - cost = 3386.07642
Epoch: 16 - cost = 7540.95459
Epoch: 16 - cost = 6917.34521
Epoch: 16 - cost = 6922.81348
Epoch: 16 - cost = 7343.59668
Epoch: 16 - cost = 6602.48633
Epoch: 16 - cost = 5082.84424
Epoch: 16 - cost = 5375.15820
Epoch: 16 - cost = 4001.69556
Epoch: 16 - cost = 3350.26318
Epoch: 16 - cost = 7551.21045
Epoch: 16 - cost = 6912.78174
Epoch: 16 - cost = 6917.93018
Epoch: 16 - cost = 7339.45752
Epoch: 16 - cost = 6562.54199
Epoch: 16 - cost = 5065.11816
Epoch: 16 - cost = 5493.84375
Epoch: 16 - cost = 3962.17139
Epoch: 16 - cost = 3383.73364
Epoch: 16 - cost = 7540.44580
Epoch: 16 - cost = 6914.39111
Epoch: 16 - cost = 6923.92920
Running epoch 16
Epoch: 17 - cost = 7343.59326
Epoch: 17 - cost = 6593.79297
Epoch: 17 - cost = 5077.07520
Epoch: 17 - cost = 5372.23828
Epoch: 17 - cost = 3996.10010
Epoch: 17 - cost = 3347

KeyboardInterrupt: 

In [22]:
saver = tf.train.Saver()

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

saver.restore(sess, './Autoencoder_Model/Autoencoder.ckpt-19')

# For reading the file contents. 
import glob as gl 

test_data = [];
for i in range(46,51):
    test_data.append(np.array(cv2.imread('./ValidationHazy/'+str(i)+'.png')))

print("All files appended")    

test_dataset = np.asarray(test_data)
print(test_dataset.shape)

# Running the test data on the autoencoder
batch_imgs = test_dataset
trans_maps = sess.run(ae_outputs, feed_dict = {ae_inputs: batch_imgs})

print("Session running succesful")

for i in range(trans_maps.shape[0]):
    print("Writing")
    cv2.imwrite('./Auto_Res_Valid_Trans/' +str(i+46) +'.png', trans_maps[i])

INFO:tensorflow:Restoring parameters from ./Autoencoder_Model/Autoencoder.ckpt-19
All files appended
(5, 1200, 1600, 3)
Session running succesful
Writing
Writing
Writing
Writing
Writing
